<a href="https://colab.research.google.com/github/Xiaoyu-Su-20/Data-Visualization-/blob/main/Xiaoyu_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [296]:
# colab 
from google.colab import drive
drive.mount("/content/drive")

# processing
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import datetime
from collections import defaultdict

# plot 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [297]:
children_3_df = pd.read_csv('/content/drive/My Drive/Datathon/data/Children_Tested_for_Lead_by_Age_3.csv')
children_6_df = pd.read_csv('/content/drive/My Drive/Datathon/data/Children_Under_6_yrs_with_Elevated_Blood_Lead_Levels__BLL_.csv')

# 3 Year Children Table

In [298]:
children_3_df.head()

,geo_type,geo_area_id,geo_area_name,borough_id,time_period,Children tested for lead by age 3 years Number,Children tested for lead by age 3 years Number _NOTES,Children tested for lead by age 3 years Percentage,Children tested for lead by age 3 years Percentage _NOTES
0,Borough,1,Bronx,1.0,2008,17500,NaN,86,NaN
1,Borough,1,Bronx,1.0,2009,18000,NaN,87,NaN
2,Borough,1,Bronx,1.0,2010,17700,NaN,87,NaN
3,Borough,1,Bronx,1.0,2011,19400,NaN,88,NaN
4,Borough,1,Bronx,1.0,2012,19000,NaN,87,NaN


### Transform children_3_df function

In [299]:
def transform_children_3(df):
  if len(df.columns) < 9: return df
  rename_cols = df.columns[:-4].tolist() + ['lead_num', 'lead_num_notes', 'lead_percentage', 'lead_percentage_notes']
  df.columns = rename_cols 
  # drop  empty column
  df.drop(['lead_num_notes', 'lead_percentage_notes'], axis=1, inplace=True)

  return df

### Separate Citywide, Borough, Neighborhood

In [300]:
children_3_df = transform_children_3(children_3_df)

In [301]:
children_3_city = children_3_df.query('geo_type == "Citywide"')
children_3_borough = children_3_df.query('geo_type == "Borough"')
children_3_neighborhood = children_3_df.query("geo_type.str.contains('Nei')", engine='python')

In [302]:
children_3_neighborhood.sort_values(by='lead_num', ascending=False)

,geo_type,geo_area_id,geo_area_name,borough_id,time_period,lead_num,lead_percentage
318,Neighborhood (UHF 42),402,West Queens,4.0,2011,6530,87
387,Neighborhood (UHF 42),402,West Queens,4.0,2013,6440,88
344,Neighborhood (UHF 42),402,West Queens,4.0,2012,6380,87
326,Neighborhood (UHF 42),402,West Queens,4.0,2015,6260,88
299,Neighborhood (UHF 42),402,West Queens,4.0,2014,6180,88
...,...,...,...,...,...,...,...
89,Neighborhood (UHF 42),310,Lower Manhattan,3.0,2009,380,62
327,Neighborhood (UHF 42),404,Bayside - Little Neck,4.0,2009,370,78
364,Neighborhood (UHF 42),310,Lower Manhattan,3.0,2010,370,58
338,Neighborhood (UHF 42),404,Bayside - Little Neck,4.0,2016,340,74


There are 9 time periods, from 2008-2016, 48 areas in total. Thus 9*48 = 432 rows make sense.

The information of `Neighborhood (UHF 42)` can be found here: http://a816-dohbesp.nyc.gov/IndicatorPublic/EPHTPDF/uhf42.pdf
42 neighborhoods, 378 total counts, over a 9 year span. 

borough_id

5 different boroughs
https://en.wikipedia.org/wiki/Boroughs_of_New_York_City

## Neighborhood EDA

In [303]:
def plot_by_year(df, start_year, num, x_col, y_col):
  fig = make_subplots(rows=num, cols=1)
  for i in range(num):
    data = df.query('time_period == {}'.format(start_year+i)).sort_values(by=y_col)
    fig.add_trace(
        go.Bar(x=data[x_col], y=data[y_col], name='{}'.format(start_year+i)),
        row=i+1, col=1
    )  

  return fig


fig = plot_by_year(children_3_neighborhood, 2008, 9, x_col='geo_area_name', y_col='lead_percentage')
fig.update_layout(height=5000, width=1200, title_text="Lead Percentage")
fig.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell', color='black', size=14))

## City EDA

In [304]:
children_3_city

,geo_type,geo_area_id,geo_area_name,borough_id,time_period,lead_num,lead_percentage
45,Citywide,1,New York City,NaN,2008,87500,80
46,Citywide,1,New York City,NaN,2009,89700,81
47,Citywide,1,New York City,NaN,2010,87300,81
48,Citywide,1,New York City,NaN,2011,98400,83
49,Citywide,1,New York City,NaN,2012,96800,83
50,Citywide,1,New York City,NaN,2013,94900,83
51,Citywide,1,New York City,NaN,2014,92800,83
52,Citywide,1,New York City,NaN,2015,91800,81
53,Citywide,1,New York City,NaN,2016,89300,81


In [305]:
# overall weighted percentage change 
# some areas have higher population, adjust the weights 
children_3_city.sort_values('time_period', inplace=True)

fig = px.bar(data_frame=children_3_city, x='time_period', y='lead_percentage', labels={'time_period':'year', 'lead_percentage':'lead percentage'})
fig.update_layout(title_text="New York City Lead Percentage From 2008 to 2016 ")

# 6 Year Children Table

### Transform children6 function

In [306]:
def transform_children_6(df):
  if len(df.columns) < 18: return df
  children_6_df.drop(children_6_df.columns[[6,8,10,12,14,16,18]], axis=1, inplace=True)
  rename_cols = children_6_df.columns[:5].tolist() + ['>=5 num','>=10 num', '>=15 num', 'number_tested', '>=5 per 1000 tested', '>=10 per 1000 tested', '>=15 per 1000 tested']
  children_6_df.columns = rename_cols
  return df

### Separate Citywide, Borough, Neighborhood

In [307]:
# drop unnecessary columns 
children_6_df = transform_children_6(children_6_df)

children_6_city = children_6_df.query('geo_type == "Citywide"')
children_6_borough = children_6_df.query('geo_type == "Borough"')
children_6_neighborhood = children_6_df.query("geo_type.str.contains('Nei')", engine='python')

In [308]:
children_6_neighborhood.query("geo_area_name.str.contains('Greenpoint')", engine='python').sort_values('time_period').head(3)

,geo_type,geo_area_id,geo_area_name,borough_id,time_period,>=5 num,>=10 num,>=15 num,number_tested,>=5 per 1000 tested,>=10 per 1000 tested,>=15 per 1000 tested
360,Neighborhood (UHF 42),201,Greenpoint,2.0,2005,869,71,22,5100,171.1,14.0,4.3
121,Neighborhood (UHF 42),201,Greenpoint,2.0,2006,786,78,23,4600,169.3,16.8,5.0
144,Neighborhood (UHF 42),201,Greenpoint,2.0,2007,864,94,19,4700,182.8,19.9,4.0


##  Neighborhood EDA

In [309]:
fig = plot_by_year(children_6_neighborhood, 2005, 12, x_col='geo_area_name', y_col='>=5 per 1000 tested')
fig.update_layout(height=5000, width=1200, title_text="Lead Percentage")
fig.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell', color='black', size=14))
fig.show()

# Residental Table

## Transform Residential function

In [310]:
def transform_residential(df):
  if len(df.columns) < 10: return df
  # drop useless columns 
  drop_col = [0, 4, 7, 10]
  # drop
  df.drop(df.columns[drop_col], axis=1, inplace=True)
  # parse dates
  df['Date Collected'] = pd.to_datetime(df['Date Collected']).dt.year
  df.rename(columns={'Date Collected':'time_period'},inplace=True) # rename 
  # change dtype
  df['Zipcode'] = df.Zipcode.astype('str')

  df = df[df['Lead First_Draw (mg/L)'] < 1]

  return df 

## Residential Lead EDA

In [311]:
Residential_df = pd.read_csv('/content/drive/My Drive/Datathon/data/Free_Residential_at-the-tap_Lead_and_Copper_Data.csv')

transform_residential(Residential_df)

,Borough,Zipcode,time_period,Lead First_Draw (mg/L),Lead 1-2 Minute Flush (mg/L),Copper First Draw (mg/L),Copper 1-2 Minute Flush (mg/L)
0,NEW YORK,10128,2014,0.000,0.000,0.051,0.095
1,BROOKLYN,11205,2014,0.000,0.000,0.224,0.056
2,BROOKLYN,11205,2014,0.000,0.000,0.279,0.240
3,NEW YORK,10028,2014,0.007,0.000,0.128,0.017
4,LONG ISLAND CITY,11101,2014,0.000,0.000,0.189,0.063
...,...,...,...,...,...,...,...
19018,QUEENS,11373,2019,0.002,0.000,0.286,0.236
19019,BROOKLYN,11218,2019,0.002,0.005,0.211,0.122
19020,QUEENS,11374,2019,0.002,0.001,0.098,0.027
19021,BROOKLYN,11215,2019,0.000,0.000,0.228,0.107


The time span is from 2014-2019, 6 years in total.

In [312]:
ZIPCODE = 10030
YEAR = 2017
Residential_df.query('Zipcode == "{}" and time_period == {}'.format(ZIPCODE, YEAR))

,Borough,Zipcode,time_period,Lead First_Draw (mg/L),Lead 1-2 Minute Flush (mg/L),Copper First Draw (mg/L),Copper 1-2 Minute Flush (mg/L)
8147,NEW YORK,10030,2017,0.000,0.000,0.118,0.048
8198,NEW YORK,10030,2017,0.160,0.003,0.098,0.007
8693,NEW YORK,10030,2017,0.129,0.003,0.126,0.010
8846,NEW YORK,10030,2017,0.000,0.000,0.182,0.163
10425,NEW YORK,10030,2017,0.013,0.003,0.192,0.180


In [313]:
def get_Residential_count(df, zipcode):
  count_df = df.Zipcode.value_counts().to_frame().reset_index()
  count_df.columns = ['Zipcode', 'count']
  return count_df.query("Zipcode == '{}'".format(zipcode))
  
get_Residential_count(Residential_df, 11222)

,Zipcode,count
9,11222,331


In [314]:
# group by zipcode and year 
aggregated_year_Residential_df = Residential_df.groupby(['Zipcode','time_period']).agg(
    avg_lead_first_draw=('Lead First_Draw (mg/L)', 'mean'),  
    avg_lead_1_2_flush=('Lead 1-2 Minute Flush (mg/L)', 'mean'),  
    avg_copper_first_draw=('Copper First Draw (mg/L)', 'mean'),  
    avg_copper_1_2_flush=('Copper 1-2 Minute Flush (mg/L)', 'mean')
).reset_index()

# don't know why but Zipcode is  continuous 
aggregated_year_Residential_df['Zipcode'] = aggregated_year_Residential_df['Zipcode'] + 'n'

In [315]:
aggregated_year_Residential_df.query('Zipcode == "{}"'.format('11357n'))

,Zipcode,time_period,avg_lead_first_draw,avg_lead_1_2_flush,avg_copper_first_draw,avg_copper_1_2_flush
763,11357n,2014,0.001143,0.002714,0.143143,0.088286
764,11357n,2015,0.001667,0.000000,0.096667,0.017333
765,11357n,2016,0.000882,0.000588,0.141000,0.049176
766,11357n,2017,0.000500,0.000750,0.209375,0.083875
767,11357n,2018,0.027462,0.009462,0.251308,0.066923
768,11357n,2019,0.002300,0.001000,0.146400,0.066200


In [316]:
# plot 
fig = plot_by_year(aggregated_year_Residential_df, 2014, 6, x_col='Zipcode', y_col='avg_copper_first_draw')
fig.update_layout(height=1500, width=1200, title_text=col_name)
fig.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell', color='black', size=14))
fig.show()

## Copper EDA

In [317]:
aggregated_year_Residential_df.head()

,Zipcode,time_period,avg_lead_first_draw,avg_lead_1_2_flush,avg_copper_first_draw,avg_copper_1_2_flush
0,10000n,2016,0.000000,0.000000,0.022000,0.004000
1,10000n,2017,0.002667,0.000000,0.146800,0.075133
2,10000n,2019,0.002000,0.001000,0.093000,0.022000
3,10001n,2014,0.007667,0.000000,0.385333,0.038000
4,10001n,2016,0.000762,0.000524,0.153952,0.074143


In [318]:
aggregated_all_Residential_df = aggregated_year_Residential_df.groupby('Zipcode').mean().drop('time_period', axis=1).reset_index()
aggregated_all_Residential_df['Zipcode'] = aggregated_all_Residential_df.Zipcode.astype('str') + 'n'

In [319]:
aggregated_all_Residential_df.sort_values(by='avg_lead_first_draw', inplace=True)

px.bar(data_frame=aggregated_all_Residential_df, x='Zipcode', y='avg_lead_first_draw')

# Compliance Table

In [320]:
compliance_df = pd.read_csv('/content/drive/My Drive/Datathon/data/Compliance_at-the-tap_Lead_and_Copper_Data.csv')

## Transform Compliance Functions

In [321]:
def count_distinct_num_for_every_col(df):
  return df.apply(lambda x:x.nunique())


def transform_compliance(df):
  if len(df.columns) < 7: return df
  drop_cols = [0, 4]
  df.drop(df.columns[drop_cols], axis='columns', inplace=True)

  df['Date Collected'] = pd.to_datetime(df['Date Collected']).dt.year
  df.rename(columns={'Date Collected':'time_period'},inplace=True) # rename 

  # change unit
  df['First Draw at-the-tap Lead level (µg/l)'] = df['First Draw at-the-tap Lead level (µg/l)'] / 1000 
  
  rename_cols = df.columns[:3].tolist() + ['Lead_First_Draw', 'Copper_First_Draw']
  df.columns = rename_cols

  return df 

## Compliance EDA

In [322]:
count_distinct_num_for_every_col(compliance_df)

KIT ID Number                                2460
Borough                                        48
Zipcode                                       110
Date Collected                                450
Received Date                                 306
First Draw at-the-tap Lead level (µg/l)        80
First Draw at-the-tap copper level (mg/l)     306
dtype: int64

In [323]:
compliance_df = transform_compliance(compliance_df)

# eliminate outliers
compliance_df = compliance_df.query('Copper_First_Draw < 1')
compliance_df = compliance_df.query('Lead_First_Draw < 1')

In [324]:
compliance_df.head()

,Borough,Zipcode,time_period,Lead_First_Draw,Copper_First_Draw
0,RIDGEWOOD,11385,2014,0.019,0.089
1,STATEN ISLAND,10306,2014,0.001,0.145
2,STATEN ISLAND,10305,2014,0.000,0.051
3,New York,10010,2014,0.000,0.143
4,MASPETH,11378,2014,0.004,0.107


In [325]:
# group by zipcode and year 
aggregated_compliance_year_df = compliance_df.groupby(['Zipcode','time_period']).agg(
    avg_lead_first_draw=('Lead_First_Draw', 'mean'),  
    avg_copper_first_draw=('Copper_First_Draw', 'mean')
).reset_index()

# don't know why but Zipcode is  continuous 
aggregated_compliance_year_df['Zipcode'] = aggregated_compliance_year_df['Zipcode'].astype('str') + 'n'

In [326]:
aggregated_compliance_year_df.query('Zipcode == "{}"'.format('11225n'))

,Zipcode,time_period,avg_lead_first_draw,avg_copper_first_draw
290,11225n,2015,0.017000,0.123000
291,11225n,2016,0.023667,0.140667
292,11225n,2017,0.034500,0.158500
293,11225n,2018,0.008000,0.128250
294,11225n,2019,0.155000,0.109500


In [332]:
y_col = 'avg_lead_first_draw'

fig = plot_by_year(aggregated_compliance_year_df, 2014, 6, x_col='Zipcode', y_col=y_col)
fig.update_layout(height=1500, width=1500, title_text=y_col)
fig.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell', color='black', size=14))
fig.show()

# Folium

In [328]:
import folium
import json

In [329]:
def get_compliance_year(df, year):
  # 
  df_needed = df.query('time_period == {}'.format(year))
  # Note: there is an 'n' cause I can't figure out how to plot discrete x scale for zipcode 
  # but even if no 'n', this step doesn't hurt
  df_needed['Zipcode'] = df_needed['Zipcode'].str.replace('n', '')  

  return df_needed 

In [330]:
YEAR = 2018
col = 'avg_copper_first_draw'

df_for_folium = get_compliance_year(aggregated_compliance_year_df, YEAR)

zipcode_geo_info = r'/content/drive/My Drive/Datathon/data/nyc_zipcode.json'

nyc_map = folium.Map(location=[40.693943, -73.985880], zoom_start=11, tiles='cartodbpositron')
folium.Choropleth(
    geo_data=zipcode_geo_info,
    name='compliace {}'.format(YEAR),
    data=df_for_folium,
    columns=['Zipcode', col],
    key_on='feature.properties.postalCode',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    highlight = True,
    legend_name='{}'.format(YEAR),
).add_to(nyc_map)

YEAR = 2019

df_for_folium = get_compliance_year(aggregated_compliance_year_df, YEAR)

folium.Choropleth(
    geo_data=zipcode_geo_info,
    name='compliace {}'.format(YEAR),
    data=df_for_folium,
    columns=['Zipcode', col],
    key_on='feature.properties.postalCode',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='{}'.format(YEAR),
).add_to(nyc_map)

folium.TileLayer('cartodbdark_matter').add_to(nyc_map)
folium.LayerControl().add_to(nyc_map)

In [331]:
nyc_map